# Poisson Modeling of Suicide Quantities vs. Cancer

The idea is to model the suicides quantity as if it had a Poisson distribution against cancer occurences.

Poisson distribution admits only integer outputs. So, we won't use the suicide/occurences rates here.

To compensate, we set the municipality's population as a parameter to the model.

We want to verify if this model fits well to the data. If that's the case, we could model the suicide quantities as a Poisson distribution, and future predictions should be worth.

References:
- https://www.nejm.org/doi/full/10.1056/NEJMoa1110307
- https://towardsdatascience.com/an-illustrated-guide-to-the-poisson-regression-model-50cccba15958

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

pd.set_option('display.max_columns', 100)
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Import datasets:

In [2]:
suicides = pd.read_csv('../Suicide/CSV/suicides_codmunres.csv')
population = pd.read_csv('../util/POPULATION_2017.csv', index_col=0)
municipalities = pd.read_csv('../util/CADMUN.csv')
cancer = pd.read_csv('CANCERBR2017.csv', index_col=0)

In [3]:
suicides.shape, population.shape, municipalities.shape, cancer.shape

((3149, 2), (5570, 2), (5652, 28), (5359, 2))

In [4]:
len(suicides[suicides['SUICIDES'] == 0])

0

Let's merge the files:

In [5]:
df = pd.merge(left=population, right=municipalities, left_on='MUNCOD', right_on='MUNCOD', how='left')
df = pd.merge(left=df, right=cancer, left_on='MUNCOD', right_on='MUNCOD', how='right')
df = pd.merge(left=df, right=suicides, left_on='MUNCOD', right_on='CODMUNRES', how='left')
df.shape

(5359, 32)

In [6]:
df.head()

,POPULATION,MUNCOD,MUNCODDV,SITUACAO,MUNSINP,MUNSIAFI,MUNNOME,MUNNOMEX,OBSERV,MUNSINON,MUNSINONDV,AMAZONIA,FRONTEIRA,CAPITAL,UFCOD,MESOCOD,MICROCOD,MSAUDCOD,RSAUDCOD,CSAUDCOD,RMETRCOD,AGLCOD,ANOINST,ANOEXT,SUCESSOR,LATITUDE,LONGITUDE,ALTITUDE,AREA,QUANTIDADE,CODMUNRES,SUICIDES
0,25437.0,110001.0,1100015.0,ATIVO,26016.0,33.0,Alta Floresta D'Oeste,ALTA FLORESTA D'OESTE,NaN,NaN,NaN,S,S,N,11.0,1102.0,11006.0,1190.0,1102.0,11900.0,1190.0,1190.0,1986.0,NaN,NaN,-11.929,-61.996,350.0,7066.702,25,110001.0,3.0
1,107345.0,110002.0,1100023.0,ATIVO,26004.0,7.0,Ariquemes,ARIQUEMES,NaN,NaN,NaN,S,N,N,11.0,1102.0,11003.0,1190.0,1104.0,11900.0,1190.0,1190.0,1977.0,NaN,NaN,-9.913,-63.041,142.0,4426.558,66,110002.0,10.0
2,6224.0,110003.0,1100031.0,ATIVO,26020.0,37.0,Cabixi,CABIXI,NaN,NaN,NaN,S,S,N,11.0,1102.0,11008.0,1190.0,1103.0,11900.0,1190.0,1190.0,1989.0,NaN,NaN,-13.492,-60.545,230.0,1314.355,4,NaN,NaN
3,88507.0,110004.0,1100049.0,ATIVO,26007.0,9.0,Cacoal,CACOAL,NaN,NaN,NaN,S,N,N,11.0,1102.0,11006.0,1190.0,1102.0,11900.0,1190.0,1190.0,1977.0,NaN,NaN,-11.438,-61.448,200.0,3792.638,133,110004.0,5.0
4,17934.0,110005.0,1100056.0,ATIVO,26014.0,27.0,Cerejeiras,CEREJEIRAS,NaN,NaN,NaN,S,S,N,11.0,1102.0,11008.0,1190.0,1103.0,11900.0,1190.0,1190.0,1983.0,NaN,NaN,-13.189,-60.812,277.0,2783.305,15,110005.0,1.0


In [7]:
df = df[['MUNCOD', 'POPULATION', 'CAPITAL', 'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'AREA', 'QUANTIDADE', 'SUICIDES']]
df = df.rename(columns={"QUANTIDADE": "CANCER_QTT"})
df.head()

,MUNCOD,POPULATION,CAPITAL,LATITUDE,LONGITUDE,ALTITUDE,AREA,CANCER_QTT,SUICIDES
0,110001.0,25437.0,N,-11.929,-61.996,350.0,7066.702,25,3.0
1,110002.0,107345.0,N,-9.913,-63.041,142.0,4426.558,66,10.0
2,110003.0,6224.0,N,-13.492,-60.545,230.0,1314.355,4,NaN
3,110004.0,88507.0,N,-11.438,-61.448,200.0,3792.638,133,5.0
4,110005.0,17934.0,N,-13.189,-60.812,277.0,2783.305,15,1.0


In [8]:
(df['POPULATION'].isnull()).value_counts()

False    5354
True        5
Name: POPULATION, dtype: int64

In [9]:
df = df[~df['POPULATION'].isnull()]
(df['POPULATION'].isnull()).value_counts()

False    5354
Name: POPULATION, dtype: int64

As there's no municipality with 0 suicides, let's assume these ones have 0 suicides for the year of 2017.

In [10]:
df['MUNCOD'] = df['MUNCOD'].astype(np.int64)
df['POPULATION'] = df['POPULATION'].astype(np.int64)
df['SUICIDES'] = df['SUICIDES'].fillna(0).astype(np.int64)

In [11]:
df.head()

,MUNCOD,POPULATION,CAPITAL,LATITUDE,LONGITUDE,ALTITUDE,AREA,CANCER_QTT,SUICIDES
0,110001,25437,N,-11.929,-61.996,350.0,7066.702,25,3
1,110002,107345,N,-9.913,-63.041,142.0,4426.558,66,10
2,110003,6224,N,-13.492,-60.545,230.0,1314.355,4,0
3,110004,88507,N,-11.438,-61.448,200.0,3792.638,133,5
4,110005,17934,N,-13.189,-60.812,277.0,2783.305,15,1


To construct a **Baseline**, let's take the estimation of 2016, when the suicide rate was 10.6 suicides per 100,000 persons [FAZEL S., "Suicide"].

In [12]:
baseline = np.floor(df['POPULATION'] / 100000 * 10.6)
baseline.head()

0     2.0
1    11.0
2     0.0
3     9.0
4     1.0
Name: POPULATION, dtype: float64

As the metrics to evaluate the baseline, we'll use:

- **Root Mean Squared Log Error (RMSLE):** Robust error metric for regression, with large error for underestimation.
- **R-Squared and Adjusted R^2 (R^2 and Adj. R^2):** Compares how well the model predicts compared to the average prediction (adjusted: not sensible to the qtt of features).

Evaluanting the baseline:

In [13]:
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score
np.sqrt(mean_squared_log_error(df['SUICIDES'], baseline)), r2_score(df['SUICIDES'], baseline)

(0.6116856140218698, -1.5781863811916517)

In [14]:
regression_params = sm.OLS(df['SUICIDES'], baseline).fit()
regression_params.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               SUICIDES   R-squared (uncentered):                   0.892
Model:                            OLS   Adj. R-squared (uncentered):              0.892
Method:                 Least Squares   F-statistic:                          4.405e+04
Date:                Sat, 02 May 2020   Prob (F-statistic):                        0.00
Time:                        23:36:03   Log-Likelihood:                         -13798.
No. Observations:                5354   AIC:                                  2.760e+04
Df Residuals:                    5353   BIC:                                  2.760e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
POPULATION     0.3825      0.002    209.876      0.000       0.379       0.386
==============================================================================
Omnibus:                     5147.535   Durbin-Watson:                   1.733
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3270942.554
Skew:                           3.725   Prob(JB):                         0.00
Kurtosis:                     123.859   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Let's try a first model:

As our examples are not so numerous (=~ 5300), we can divide the dataset as follows:

- 50% training set
- 50% validation set

In [16]:
from sklearn.model_selection import KFold

X = df[['POPULATION', 'CANCER_QTT']] #df[df.columns[:-1]]
y = df['SUICIDES']

rmsle_vector = []
r2_vector = []

for seed in range(10):
    kf = KFold(n_splits=2, random_state=seed, shuffle=True)
    
    for train_index, test_index in kf.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        print()
        poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
        #print(poisson_training_results.summary())
        poisson_predictions = poisson_training_results.get_prediction(X_test)
        predictions_summary_frame = poisson_predictions.summary_frame()
        predicted_counts = predictions_summary_frame['mean']

        rmsle = np.sqrt(mean_squared_log_error(y_test, predicted_counts))
        r2 = r2_score(y_test, predicted_counts)
        rmsle_vector.append(rmsle)
        r2_vector.append(r2)
        print("RMSLE: {}".format(rmsle))
        print("R^2: {}".format(r2))

        print("\n==========\n")

print()
print("Summary:\n")
print("Average RMSLE: {}".format(np.mean(rmsle_vector)))
print("Average R^2: {}".format(np.mean(r2_vector)))

TRAIN: [   0    6    7 ... 5348 5351 5353] TEST: [   1    2    3 ... 5349 5350 5352]

RMSLE: 0.7485299607453743
R^2: 0.15759071297167937


TRAIN: [   1    2    3 ... 5349 5350 5352] TEST: [   0    6    7 ... 5348 5351 5353]

RMSLE: 0.7421354510648148
R^2: -39917.687952715976


TRAIN: [   0    2    8 ... 5347 5350 5353] TEST: [   1    3    4 ... 5349 5351 5352]

RMSLE: 0.8144330438454526
R^2: -1149576677233417.5


TRAIN: [   1    3    4 ... 5349 5351 5352] TEST: [   0    2    8 ... 5347 5350 5353]

RMSLE: 0.7398721040927421
R^2: 0.016419684766867615


TRAIN: [   0    1    6 ... 5348 5351 5352] TEST: [   2    3    4 ... 5349 5350 5353]

RMSLE: 1.0009308424421102
R^2: -3.4526710806141316e+27


TRAIN: [   2    3    4 ... 5349 5350 5353] TEST: [   0    1    6 ... 5348 5351 5352]

RMSLE: 0.7466189685006035
R^2: 0.13550222465946737


TRAIN: [   1    2    4 ... 5348 5351 5353] TEST: [   0    3    6 ... 5349 5350 5352]

RMSLE: 0.7447205829222417
R^2: -0.03350082790761455


TRAIN: [   0    3    

As we see, our metrics show greater error (RMSLE) and inferior R^2.

So, this first Poisson model is **inferior to the chosen baseline.**

---

Next steps:

- Model as Negative Binomial Regression (doesn't imply that the mean and variance are equal)

- Consider other variables in the model (CAPITAL, LATITUDE, LONGITUDE, ALTITUDE, AREA) in order to understand better the data (don't forget to cast to positive numbers)

- Try with other diseases